In [ ]:
import numpy as np 
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
%tensorflow_version 1.15

#print(keras.__version__)

if (tf.__version__.split('.')[0] == '2'):
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()

# Load MNIST dataset
import input_data

# Load MNIST dataset
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  # call mnist function

learningRate = 1e-3
trainingIters = 150000
batchSize = 50
displayStep = 1000

nInput = 28  # we want the input to take the 28 pixels
nSteps = 28  # every 28
nHidden = 50  # number of neurons for the RNN
nClasses = 10  # this is MNIST so you know

x = tf.placeholder('float', [None, nSteps, nInput])
y = tf.placeholder('float', [None, nClasses])

weights = {
    'out': tf.Variable(tf.random_normal([nHidden, nClasses]))
}

biases = {
    'out': tf.Variable(tf.random_normal([nClasses]))
}


def RNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, nInput])
    x = tf.split(x, nSteps, 0)  # configuring so you can get it as needed for the 28 pixels

    # find which lstm to use in the documentation
    # lstmCell = rnn_cell.BasicLSTMCell(nHidden, forget_bias=1.0)
    # lstmCell = rnn_cell.GRUCell(nHidden)
    lstmCell = rnn_cell.BasicRNNCell(num_units = nHidden)

    outputs, states = rnn.static_rnn(lstmCell, x,
                                     dtype=tf.float32)  # for the rnn where to get the output and hidden state

    return tf.matmul(outputs[-1], weights['out']) + biases['out']


pred = RNN(x, weights, biases)

# optimization
# create the cost, optimization, evaluation, and accuracy
# for the cost softmax_cross_entropy_with_logits seems really good
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learningRate).minimize(cost)

correctPred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

init = tf.initialize_all_variables()

loss_list = []
accuracy_list = []

with tf.Session() as sess:
    sess.run(init)
    step = 1

    while step * batchSize < trainingIters:
        batchX, batchY = mnist.train.next_batch(batchSize)  # mnist has a way to get the next batch
        batchX = batchX.reshape((batchSize, nSteps, nInput))

        sess.run(optimizer, feed_dict={x: batchX, y: batchY})

        acc = sess.run(accuracy, feed_dict={x: batchX, y: batchY})
        loss = sess.run(cost, feed_dict={x: batchX, y: batchY})

        loss_list.append(loss)
        accuracy_list.append(acc)

        if step % displayStep == 0:
            print("Iter " + str(step * batchSize) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print('Optimization finished')

    testData = mnist.test.images.reshape((-1, nSteps, nInput))
    testLabel = mnist.test.labels
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: testData, y: testLabel}))
    print("Testing loss:", sess.run(cost, feed_dict={x: testData, y: testLabel}))
    sess.close()

# Plot the accuracy and loss under different parameters
plt.figure(figsize=(12, 9))
plt.subplot(1, 2, 1)
plt.plot(range(len(accuracy_list)), accuracy_list, label='Accuracy')
plt.title('Accuracy_RNN_50_neuron')
plt.legend()
plt.xlabel('Steps')
plt.ylabel('Accuracy')
plt.ylim(0, 1.05)

plt.subplot(1, 2, 2)
plt.plot(range(len(loss_list)), loss_list, label='Loss')
plt.title('Loss_RNN_50_neuron')
plt.legend()
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.show()
